In [2]:
#!pip install newspaper3k
#!pip install lxml[html_clean]
#!pip install together
#!pip install sendgrid
#!pip install newsapi-python

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=990d0e7aabb02332cc184de019cf307294eac9faa24fb36fa38e9065d57eebbb
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=fdbbe945aa23aea75da336db21b2980215d0c3f7ec21ca60b418fd6ff3cd9581
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

In [49]:
from datetime import datetime

import getpass

from IPython.core.display import HTML

from newsapi import NewsApiClient

from newspaper import Article

import requests

import textwrap

from together import Together

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
from sendgrid import SendGridAPIClient


In [50]:
def get_audience_level():

    """
    Prompts the user to select an audience level and returns the corresponding level number.

    Args:
        None

    Returns:
        int: An integer (0-4) representing the selected audience level.

    Audience Levels:
        0: Basic level (Popular Science)
        1: Intermediate level (Casual but Informative)
        2: Technical level (For AI Enthusiasts and Students)
        3: Advanced level (For Professionals and Developers)
        4: Expert level (AI Research and Development)

    Raises:
        ValueError: If the user enters a non-integer value.
    """

    levels = {
        0: "basic level (Popular Science)",
        1: "intermediate level (Casual but Informative)",
        2: "technical level (For AI Enthusiasts and Students)",
        3: "advanced level (For Professionals and Developers)",
        4: "expert level (AI Research and Development)"
    }


    try:

        audience_no = int(input(
            """
Please enter a number (0-4) corresponding to the audience level:
0 = Basic level (Popular Science),
1 = Intermediate level (Casual but Informative),
2 = Technical level (For AI Enthusiasts and Students),
3 = Advanced level (For Professionals and Developers),
4 = Expert level (AI Research and Development)
"""
        ))

        if audience_no in levels:

            print(f"You selected {levels[audience_no]}")

            return audience_no
        else:

            print("Invalid input. Please enter a number between 0 and 4.")
            return get_audience_level()

    except ValueError:

        print("Invalid input. Please enter an integer.")
        return get_audience_level()


In [51]:
def get_start_date():
    """
    Prompts the user to enter a start date and returns it as a string.

    Args:
        None

    Returns:
        str: The entered start date in the format 'YYYY-MM-DD'.

    Raises:
        ValueError: If the input is not a valid date in the expected format.
    """

    try:
        start_date_str=input("Enter the start date (YYYY-MM-DD): ")
        start_date = datetime.strptime(
            start_date_str,
            "%Y-%m-%d"
        )

        return start_date_str

    except ValueError:

        print("Invalid input. Please enter a valid date in the format YYYY-MM-DD.")
        return get_start_date()

In [52]:
def get_end_date(start_date):
    """
    Prompts the user to enter an end date and returns it as a string.

    Args:
        start_date (str): The start date in the format 'YYYY-MM-DD'.

    Returns:
        str: The entered end date in the format 'YYYY-MM-DD'.

    Raises:
        ValueError: If the input is not a valid date in the expected format.
        ValueError: If the end date is earlier than the start date.
  """

    try:
        end_date_str = input("Enter the end date (YYYY-MM-DD): ")
        end_date=datetime.strptime(
            end_date_str,
            "%Y-%m-%d"
        )

        if end_date >= datetime.strptime(start_date, "%Y-%m-%d"):
            return end_date_str
        else:
            print("The end date must be greater than or equal to the start date.")
            return get_end_date(start_date)

    except ValueError:

        print("Invalid input. Please enter a valid date in the format YYYY-MM-DD.")
        return get_end_date(start_date)

In [59]:
def create_and_send_newsletter(news_api_key,together_api_key, your_mail, sendgrid_api_key, audience_no, start_date, end_date):
    """
    Creates and sends a personalized newsletter based on AI-related news articles, summarizing them for a specified audience.

    Args:
        news_api_key (str): API key for accessing the news API.
        together_api_key (str): API key for accessing the Together API to summarize content.
        your_mail (str): Email address of the sender for the newsletter.
        sendgrid_api_key (str): API key for sending the email through SendGrid.
        audience_no (int): The audience level (0 to 4) for tailoring the newsletter content.
        start_date (str): The start date for fetching news in 'YYYY-MM-DD' format.
        end_date (str): The end date for fetching news in 'YYYY-MM-DD' format.

    Returns:
        str: The HTML content of the created newsletter.

    Raises:
        Exception: If there is an issue with the APIs or while sending the email.
    """

    # GET News
    results_limit = 100

    queries = [
        "'artificial intelligence'", "'machine learning'", "'deep learning'",
        "'NVIDIA'", "'OpenAI'", "'Claude'", "'GPT'", "'LLM'", "'ChatGPT'", "'AI'",
        "'neural networks'", "'quantum'", "'TensorFlow'", "'Google'", "'Deepseek'",
        "'NLP'", "'generative AI'", "'Hugging Face'", "'Anthropic'", "'Meta'"
    ]

    query= " OR ".join(queries)

    newsapi = NewsApiClient(api_key=news_api_key)

    try:
        all_news = newsapi.get_everything(
            q=query,
            from_param=start_date,
            to=end_date,
            language='en',
            sort_by='popularity',
            page_size=results_limit
        )

        if all_news['status'] != 'ok':

            print(f"Error: {all_news['message']}")

            return None

        all_articles = all_news['articles']

    except Exception as e:

        print(f"An error occurred with the NewsAPI: {e}")

        return None


    # Object and params for summarizing
    audiences = [
        "basic level (Popular Science)",
        "intermediate level (Casual but Informative)",
        "technical level (For AI Enthusiasts and Students)",
        "advanced level (For Professionals and Developers)",
        "expert level (AI Research and Development)"
    ]
    audience=audiences[audience_no]

    client = Together(api_key=together_api_key)


    # Build json with news content (summarized if too long)
    json_news = {}

    if len(all_articles)>0:

      for index, article in enumerate(all_articles):

            article_url = article['url']

            try:
                article_obj = Article(article_url)
                article_obj.download()
                article_obj.parse()

                if len(article_obj.text)<600:

                    json_news[str(index)] = {
                        "title": article['title'],
                        "content": article_obj.text,
                    }
                else:
                    if len(article_obj.text)<6000:
                        content_to_summarize = article_obj.text
                    else:
                        content_to_summarize = article_obj.text[:6000]

                    summarize_prompt = f"""
                        You are an expert investigator and AI developer, specialized \
                        in communication through newsletters for any type of audience, \
                        from those who require explanations at a basic level to  \
                        dissemination aimed at experts at your level.
                        You have the following news content.
                        <content>
                        {content_to_summarize}
                        </content>

                        Considering the following audience levels:
                            Basic level (Popular Science),
                            Intermediate level (Casual but Informative),
                            Technical level (For AI Enthusiasts and Students),
                            Advanced level (For Professionals and Developers),
                            Expert level (AI Research and Development)

                        Summarize the news content for an audience of {audience}.
                        The summary must have a maximum length of 600 characters
                        The summary should be clear and appropriate for the audience's \
                        level of expertise.
                        Ensure that the summary is based only on the information contained \
                        in the news content, while being as complete as possible.

                        Return only the summary.
                    """
                    try:
                        response = client.chat.completions.create(
                            model="Qwen/Qwen2-VL-72B-Instruct",
                            messages=[{"role": "user", "content": textwrap.dedent(summarize_prompt)}],
                            temperature=0.5
                        )

                        json_news[str(index)] = {
                            "title": article['title'],
                            "content": response.choices[0].message.content,
                        }

                    except Exception as e:
                        print(f"Error summarizing article {article_url} with Together API: {e}")

                        continue

            except Exception as e:
                print(f"Error processing article {article_url}: {e}")

                continue

    #Create newsletter in html
    newsletter_prompt = f"""
        You are an expert investigator and AI developer, specialized in communication \
        through newsletters for any type of audience, from those who require \
        explanations at a basic level to dissemination aimed at experts at your level.

        You have a JSON collection of news headlines and their content.
        Below is that information enclosed within <news></news>.
        <news>
        {json_news}
        </news>

        Identify the news that refer to the same story, even if they use \
        different wording.
        Prioritizing the 10 stories more repeated with different wording and \
        that are related to Artificial Intelligence.
        Each of those 10 stories should be considered as a single news and \
        should be structured with:
        - A clear, concise headline that is specific and unique.
        - An explanation of the content (1 to 3 full paragraphs). If possible,
        please include 3 paragraphs, as long as the information allows you to do so.


        Considering the following audience levels:
            Basic level (Popular Science),
            Intermediate level (Casual but Informative),
            Technical level (For AI Enthusiasts and Students),
            Advanced level (For Professionals and Developers),
            Expert level (AI Research and Development)

        The content should be summarized and explained in a way that is suitable \
        for an audience of {audience}, ensuring the explanations are appropriate \
        for their level of understanding.
        The content referring to factual updates should also remain faithful to \
        the information provided in the news articles, \
        without inferring or making assumptions about events beyond that.
        You may add technical explanations where appropriate, but stay within your \
        current knowledge level.

        The explanations should be written in 1 to 3 paragraphs, and if the \
        information is sufficient, aim to complete 3 separare paragraphs.
        Each paragraph should be a logical unit of information, containing \
        multiple sentences.

        Do not number the articles or refer to them as news or stories, simply \
        write about their content.

        Return only the body of the newsletter in HTML format:
        - For each topic, use <h1> for the headline, <p> for the explanation.
        - The content should be formatted for clarity, using simple HTML tags.
    """
    newsletter_response = client.chat.completions.create(
        model="Qwen/Qwen2-VL-72B-Instruct",
        messages=[{"role": "user", "content": textwrap.dedent(newsletter_prompt)}],
        temperature=0.5
    )

    #Send newsletter
    message = Mail(
        from_email=your_mail,
        to_emails=your_mail,
        subject='Newsletter',
        html_content=newsletter_response.choices[0].message.content)
    try:
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)

    except Exception as e:
        print(e.message)

    return newsletter_response.choices[0].message.content


In [60]:
news_api_key = getpass.getpass("Enter your newsapi.org API Key:")
together_api_key = getpass.getpass("Enter your api.together.ai API Key:")
your_mail = getpass.getpass("Enter your mail:")
sendgrid_api_key = getpass.getpass("Enter your twilio Sendgrid API Key:")
audience_no = get_audience_level()
start_date = get_start_date()
end_date = get_end_date(start_date)

display(
    HTML(
        create_and_send_newsletter(
            news_api_key,
            together_api_key,
            your_mail,
            sendgrid_api_key,
            audience_no,
            start_date,
            end_date
        )
    )
)

Enter your newsapi.org API Key:··········
Enter your api.together.ai API Key:··········
Enter your mail:··········
Enter your twilio Sendgrid API Key:··········

Please enter a number (0-4) corresponding to the audience level:
0 = Basic level (Popular Science),
1 = Intermediate level (Casual but Informative),
2 = Technical level (For AI Enthusiasts and Students),
3 = Advanced level (For Professionals and Developers),
4 = Expert level (AI Research and Development)
4
You selected expert level (AI Research and Development)
Enter the start date (YYYY-MM-DD): 2025-02-21
Enter the end date (YYYY-MM-DD): 2025-02-28
Error processing article https://venturebeat.com/ai/openai-drops-deep-research-access-to-plus-users-heating-up-ai-agent-wars-with-deepseek-and-claude/: Article `download()` failed with 403 Client Error: Forbidden for url: https://venturebeat.com/ai/openai-drops-deep-research-access-to-plus-users-heating-up-ai-agent-wars-with-deepseek-and-claude/ on URL https://venturebeat.com/ai/op